In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

# Markdown-Datei laden
with open("Extrahierter_Text_Markdown/Daimler_Truck_Holding_2023.md", "r", encoding="utf-8") as f:
    text = f.read()

def simple_financial_chunker(text, chunk_size=1000, chunk_overlap=80):
    """
    Einfacher Chunker für Finanzberichte - Tabellen werden separat behandelt
    """
    
    # 1. Tabellen extrahieren und durch Platzhalter ersetzen
    table_pattern = r'--- Tabelle Start ---(.*?)--- Tabelle Ende ---'
    tables = []
    
    for match in re.finditer(table_pattern, text, re.DOTALL):
        tables.append(match.group(1).strip())
    
    # Text ohne Tabellen
    text_without_tables = re.sub(table_pattern, '<<TABLE_PLACEHOLDER>>', text, flags=re.DOTALL)
    
    # 2. Standard Text-Chunking
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=[
            "\n\n\n",   # Große Absätze
            "\n\n",     # Normale Absätze  
            "\n",       # Zeilen
            ". ",       # Sätze
            " ",        # Wörter
            ""          # Zeichen
        ]
    )
    
    # Text chunken
    text_chunks = text_splitter.split_text(text_without_tables)
    
    # 3. Tabellen bereinigen und hinzufügen
    table_chunks = []
    for i, table_content in enumerate(tables):
        cleaned_table = clean_table_simple(table_content)
        table_chunks.append(f"📊 **TABELLE {i+1}**\n\n{cleaned_table}")
    
    return text_chunks + table_chunks

def clean_table_simple(table_text):
    """
    Einfache Tabellen-Bereinigung ohne Klassifikation
    """
    lines = [line.strip() for line in table_text.split('\n') if line.strip()]
    
    cleaned_lines = []
    for line in lines:
        if '|' in line and not line.startswith('---'):
            # Leere Zellen durch "—" ersetzen
            cells = [cell.strip() for cell in line.split('|')]
            if len(cells) > 2:  # Echte Tabellen-Zeile
                cells = [cell if cell else "—" for cell in cells]
                cleaned_lines.append('| ' + ' | '.join(cells[1:-1]) + ' |')
    
    return '\n'.join(cleaned_lines)

# Chunks erstellen
chunks = simple_financial_chunker(text, chunk_size=1000, chunk_overlap=80)

print(f"✅ {len(chunks)} Chunks erstellt")

# Aufteilung anzeigen
table_chunks = [chunk for chunk in chunks if chunk.startswith('📊')]
text_chunks = [chunk for chunk in chunks if not chunk.startswith('📊')]

print(f"📄 Text-Chunks: {len(text_chunks)}")
print(f"📊 Tabellen-Chunks: {len(table_chunks)}")

if table_chunks:
    print("\n--- BEISPIEL TABELLEN-CHUNK ---")
    print(table_chunks[0][:400] + "...")

if text_chunks:
    print("\n--- BEISPIEL TEXT-CHUNK ---")  
    print(text_chunks[0][:300] + "...")

ModuleNotFoundError: No module named 'langchain'

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

ModuleNotFoundError: No module named 'langchain'

In [21]:
with open("BMW_2023.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [22]:
text

'DRIVING THE\nBMW GROUP BERICHT 2023 NEXT ERA\nAuf dem Weg zur elektrischen und digitalisierten Mobilität in einer\nnachhaltigen Kreislaufwirtschaft. Bericht über die ökonomische Leistung\nder BMW Group und ihren ökologischen und gesellschaftlichen Beitrag.\n↗ Link zum Online-Bericht\n\n2 BMW Group Bericht 2023 An unsere Stakeholder Zusammengefasster Lagebericht Konzernabschluss Bilanzeid und Vermerke des Prüfers Vergütungsbericht Weitere Informationen\nBMW GROUP BERICHT 2023\nDie BMW Group blickt auf ein erfolgreiches Jahr 2023. Das Unternehmen wächst und ist profitabel zugleich. Die Marke BMW ist erneut die Nummer eins im Premiumsegment. Über alle Marken,\nSegmente und Antriebstechnologien bieten wir unseren Kundinnen und Kunden ein hochattraktives und innovatives Produktprogramm. Fahrzeuge mit vollelektrischen Antrieben sind dabei unser\ngrößter Wachstumstreiber. Unsere Visionsfahrzeuge zeigen gleichzeitig, wie wir die NEUE KLASSE als nächsten Innovationssprung auf den Weg bringen. 

In [23]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    separators=["\n\n", "\n", ".", " ", ""],  # Wichtig für gute Schnitte
)

In [24]:
chunks = text_splitter.split_text(text)

In [25]:
chunks

['DRIVING THE\nBMW GROUP BERICHT 2023 NEXT ERA\nAuf dem Weg zur elektrischen und digitalisierten Mobilität in einer\nnachhaltigen Kreislaufwirtschaft. Bericht über die ökonomische Leistung',
 'der BMW Group und ihren ökologischen und gesellschaftlichen Beitrag.\n↗ Link zum Online-Bericht',
 '2 BMW Group Bericht 2023 An unsere Stakeholder Zusammengefasster Lagebericht Konzernabschluss Bilanzeid und Vermerke des Prüfers Vergütungsbericht Weitere Informationen\nBMW GROUP BERICHT 2023',
 'Die BMW Group blickt auf ein erfolgreiches Jahr 2023. Das Unternehmen wächst und ist profitabel zugleich. Die Marke BMW ist erneut die Nummer eins im Premiumsegment. Über alle Marken,',
 'Segmente und Antriebstechnologien bieten wir unseren Kundinnen und Kunden ein hochattraktives und innovatives Produktprogramm. Fahrzeuge mit vollelektrischen Antrieben sind dabei unser',
 'größter Wachstumstreiber. Unsere Visionsfahrzeuge zeigen gleichzeitig, wie wir die NEUE KLASSE als nächsten Innovationssprung auf den

In [26]:
len(chunks)

7426